## Final Project



Testing with Covid care dataset and nytimes country level cumulative cases data    
https://github.com/covidcaremap/covid19-healthsystemcapacity/tree/master/data    
https://github.com/nytimes/covid-19-data

In [14]:
import pandas as pd
import matplotlib.pyplot as plt

In [15]:
covid_care_df = pd.read_csv (r'us_healthcare_capacity-county-CovidCareMap.csv')
us_counties_df = pd.read_csv(r'us-counties.csv')
capacity_state = pd.read_csv("capacity_state.csv")

In [16]:
state_abbs=capacity_state[['State','State Name']]

In [17]:
# sum across county
covid_care_df.columns = covid_care_df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
covid_care_df = covid_care_df.rename(columns={'county_name': 'county'})
covid_care_df = covid_care_df.rename(columns={'state': 'State'})
covid_care_df = covid_care_df.groupby(['county', 'State']).sum()
covid_care_df.index.name = 'county'
covid_care_df.reset_index(inplace=True)
covid_care_df = pd.merge(covid_care_df, state_abbs, how='left', on=['State'])
covid_care_df['location'] = covid_care_df['county'].str.cat(covid_care_df['State Name'], sep = ", ")
covid_care_df = covid_care_df.drop(['State', 'fips_code'], axis = 1)
covid_care_df

,county,staffed_all_beds,staffed_icu_beds,licensed_all_beds,all_bed_occupancy_rate,icu_bed_occupancy_rate,population,population_20+,population_65+,staffed_all_beds_[per_1000_people],staffed_all_beds_[per_1000_adults_20+],staffed_all_beds_[per_1000_elderly_65+],staffed_icu_beds_[per_1000_people],staffed_icu_beds_[per_1000_adults_20+],staffed_icu_beds_[per_1000_elderly_65+],licensed_all_beds_[per_1000_people],licensed_all_beds_[per_1000_adults_20+],licensed_all_beds_[per_1000_elderly_65+],State Name,location
0,Abbeville,25.0,6.0,25.0,0.32,0.03,24541.0,18884.0,5343.0,1.019,1.324,4.679,0.244,0.318,1.123,1.019,1.324,4.679,South Carolina,"Abbeville, South Carolina"
1,Acadia,205.0,4.0,186.0,0.28,0.85,62190.0,44673.0,9491.0,3.296,4.589,21.599,0.064,0.090,0.421,2.991,4.164,19.598,Louisiana,"Acadia, Louisiana"
2,Ada,1195.0,105.0,1350.0,0.62,0.55,469966.0,345855.0,67792.0,2.543,3.455,17.627,0.223,0.304,1.549,2.873,3.903,19.914,Idaho,"Ada, Idaho"
3,Adair,25.0,0.0,25.0,0.04,0.00,7063.0,5412.0,1603.0,3.540,4.619,15.596,0.000,0.000,0.000,3.540,4.619,15.596,Iowa,"Adair, Iowa"
4,Adair,49.0,0.0,77.0,0.01,0.00,19215.0,14529.0,3571.0,2.550,3.373,13.722,0.000,0.000,0.000,4.007,5.300,21.563,Kentucky,"Adair, Kentucky"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2528,York,318.0,24.0,338.0,0.74,0.69,274118.0,200604.0,39236.0,1.160,1.585,8.105,0.088,0.120,0.612,1.233,1.685,8.615,South Carolina,"York, South Carolina"
2529,Young,50.0,0.0,50.0,0.28,0.00,18045.0,13298.0,3671.0,2.771,3.760,13.620,0.000,0.000,0.000,2.771,3.760,13.620,Texas,"Young, Texas"
2530,Yuba,209.0,24.0,221.0,0.74,0.67,78041.0,54627.0,9955.0,2.678,3.826,20.994,0.308,0.439,2.411,2.832,4.046,22.200,California,"Yuba, California"
2531,Yuma,457.0,42.0,457.0,0.48,0.40,212128.0,152468.0,39828.0,2.154,2.997,11.474,0.198,0.275,1.055,2.154,2.997,11.474,Arizona,"Yuma, Arizona"


In [18]:
# select most recent date for cumulative data and sum across county
April_date = us_counties_df['date'] == '2020-04-22'
April_us_counties_df = us_counties_df[April_date]
April_us_counties_df = April_us_counties_df.drop(['fips', 'date'], 1)
April_us_counties_df = April_us_counties_df.groupby(['county', 'state']).sum()
April_us_counties_df.index.name = 'county'
April_us_counties_df.reset_index(inplace=True)
April_us_counties_df['location'] = April_us_counties_df['county'].str.cat(April_us_counties_df['state'], sep = ", ")
#April_us_counties_df = April_us_counties_df.drop(['county', 'state'], axis = 1)
April_us_counties_df

,county,state,cases,deaths,location
0,Abbeville,South Carolina,22,0,"Abbeville, South Carolina"
1,Acadia,Louisiana,117,7,"Acadia, Louisiana"
2,Accomack,Virginia,52,1,"Accomack, Virginia"
3,Ada,Idaho,597,14,"Ada, Idaho"
4,Adair,Iowa,1,0,"Adair, Iowa"
...,...,...,...,...,...
2788,Yukon-Koyukuk Census Area,Alaska,1,0,"Yukon-Koyukuk Census Area, Alaska"
2789,Yuma,Arizona,28,1,"Yuma, Arizona"
2790,Yuma,Colorado,4,0,"Yuma, Colorado"
2791,Zapata,Texas,7,0,"Zapata, Texas"


We have 1568 counties recorded in the CovidCare dataset and 1654 counties recorded in the US cases of covid-19 dataset. 

In [19]:
data = pd.merge(covid_care_df, April_us_counties_df, how = 'left', on='location')
data_final = data.drop(['county_y', 'State Name'], axis = 1)
data_final

,county_x,staffed_all_beds,staffed_icu_beds,licensed_all_beds,all_bed_occupancy_rate,icu_bed_occupancy_rate,population,population_20+,population_65+,staffed_all_beds_[per_1000_people],...,staffed_icu_beds_[per_1000_people],staffed_icu_beds_[per_1000_adults_20+],staffed_icu_beds_[per_1000_elderly_65+],licensed_all_beds_[per_1000_people],licensed_all_beds_[per_1000_adults_20+],licensed_all_beds_[per_1000_elderly_65+],location,state,cases,deaths
0,Abbeville,25.0,6.0,25.0,0.32,0.03,24541.0,18884.0,5343.0,1.019,...,0.244,0.318,1.123,1.019,1.324,4.679,"Abbeville, South Carolina",South Carolina,22.0,0.0
1,Acadia,205.0,4.0,186.0,0.28,0.85,62190.0,44673.0,9491.0,3.296,...,0.064,0.090,0.421,2.991,4.164,19.598,"Acadia, Louisiana",Louisiana,117.0,7.0
2,Ada,1195.0,105.0,1350.0,0.62,0.55,469966.0,345855.0,67792.0,2.543,...,0.223,0.304,1.549,2.873,3.903,19.914,"Ada, Idaho",Idaho,597.0,14.0
3,Adair,25.0,0.0,25.0,0.04,0.00,7063.0,5412.0,1603.0,3.540,...,0.000,0.000,0.000,3.540,4.619,15.596,"Adair, Iowa",Iowa,1.0,0.0
4,Adair,49.0,0.0,77.0,0.01,0.00,19215.0,14529.0,3571.0,2.550,...,0.000,0.000,0.000,4.007,5.300,21.563,"Adair, Kentucky",Kentucky,60.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2528,York,318.0,24.0,338.0,0.74,0.69,274118.0,200604.0,39236.0,1.160,...,0.088,0.120,0.612,1.233,1.685,8.615,"York, South Carolina",South Carolina,166.0,3.0
2529,Young,50.0,0.0,50.0,0.28,0.00,18045.0,13298.0,3671.0,2.771,...,0.000,0.000,0.000,2.771,3.760,13.620,"Young, Texas",Texas,4.0,1.0
2530,Yuba,209.0,24.0,221.0,0.74,0.67,78041.0,54627.0,9955.0,2.678,...,0.308,0.439,2.411,2.832,4.046,22.200,"Yuba, California",California,16.0,1.0
2531,Yuma,457.0,42.0,457.0,0.48,0.40,212128.0,152468.0,39828.0,2.154,...,0.198,0.275,1.055,2.154,2.997,11.474,"Yuma, Arizona",Arizona,28.0,1.0


## Baseline Methods

Exploratory data analysis and figures/graphs. Summary statistics. 

## Methods
1. model capability using multiple linear regression.  
2. Apply multivariate bootstrap techniques with hypothesis testing for model selection.  
3. Clustering (hierarchical clustering) for capability classification based on US regions.   

### Step 1: